### ВЫДЕЛЕНИЕ ИТОГОВОГО НАБОРА ПОЛЕЙ ДЛЯ ОДНОЙ ЗАПИСИ

*Загружаю необхожимые библиотеки:*

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.rcParams ["figure.figsize"]=(6,4)
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
import warnings
warnings.filterwarnings('ignore')
import nltk
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix,max_error
from xgboost import XGBClassifier, plot_importance
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import SGDClassifier
import telebot

*Згружаю данные:*

In [2]:
dataset=pd.read_csv("brooklyn_sales_map.csv")
dataset.sample(3)

,Unnamed: 0,borough,neighborhood,building_class_category,tax_class,block,lot,easement,building_class,address,...,EDesigNum,APPBBL,APPDate,PLUTOMapID,FIRM07_FLA,PFIRM15_FL,Version,MAPPLUTO_F,SHAPE_Leng,SHAPE_Area
18482,18483,3,BROOKLYN HEIGHTS,10 COOPS - ELEVATOR APARTMENTS,2,217,1,NaN,D4,"75 HENRY STREET, 28H",...,NaN,0.0,NaN,1.0,NaN,NaN,17V1.1,0.0,1928.040402,141133.679740
100021,100022,3,GREENPOINT,07 RENTALS - WALKUP APARTMENTS,2A,2706,6,NaN,C2,178 KINGSLAND AVE,...,NaN,0.0,NaN,1.0,NaN,NaN,17V1.1,0.0,192.987527,1772.779137
16221,16222,3,DOWNTOWN-METROTECH,13 CONDOS - ELEVATOR APARTMENTS,2,2058,1260,NaN,R4,365 BRIDGE STREET,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


*С помощью $shape$ смотрю размер выборки:*

In [3]:
dataset.shape

(390883, 111)

*С помощью функции $info()$, смотрю информацию о данных:*

In [4]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 390883 entries, 0 to 390882
Columns: 111 entries, Unnamed: 0 to SHAPE_Area
dtypes: float64(64), int64(11), object(36)
memory usage: 331.0+ MB


### ПРЕДОБРАБОТКА ДАННЫХ

*Смотрю процент пропущенных значений в данных:*

In [5]:
for col in dataset.columns:
    pct_missing = np.mean(dataset[col].isnull())
    print('{} - {}%'.format(col, round(pct_missing*100)))

Unnamed: 0 - 0%
borough - 0%
neighborhood - 0%
building_class_category - 0%
tax_class - 2%
block - 0%
lot - 0%
easement - 100%
building_class - 2%
address - 0%
apartment_number - 78%
zip_code - 0%
residential_units - 0%
commercial_units - 0%
total_units - 0%
land_sqft - 0%
gross_sqft - 0%
year_built - 0%
tax_class_at_sale - 0%
building_class_at_sale - 0%
sale_price - 0%
sale_date - 0%
year_of_sale - 0%
Borough - 22%
CD - 22%
CT2010 - 22%
CB2010 - 23%
SchoolDist - 22%
Council - 22%
ZipCode - 22%
FireComp - 22%
PolicePrct - 22%
HealthCent - 22%
HealthArea - 22%
SanitBoro - 22%
SanitDistr - 22%
SanitSub - 22%
Address - 22%
ZoneDist1 - 22%
ZoneDist2 - 96%
ZoneDist3 - 100%
ZoneDist4 - 100%
Overlay1 - 89%
Overlay2 - 100%
SPDist1 - 91%
SPDist2 - 100%
SPDist3 - 100%
LtdHeight - 99%
SplitZone - 22%
BldgClass - 22%
LandUse - 23%
Easements - 22%
OwnerType - 86%
OwnerName - 22%
LotArea - 22%
BldgArea - 22%
ComArea - 22%
ResArea - 22%
OfficeArea - 22%
RetailArea - 22%
GarageArea - 22%
StrgeArea - 2

*Удаляю все неинформативные признаки:*

In [6]:
del dataset["Unnamed: 0"]
del dataset["borough"]
del dataset["easement"]
del dataset["apartment_number"]
del dataset["ZoneDist2"]
del dataset["ZoneDist3"]
del dataset["ZoneDist4"]
del dataset["Overlay1"]
del dataset["Overlay2"]
del dataset["SPDist1"]
del dataset["SPDist2"]
del dataset["SPDist3"]
del dataset["LtdHeight"]
del dataset["OwnerType"]
del dataset["Ext"]
del dataset["HistDist"]
del dataset["Landmark"]
del dataset["ZMCode"]
del dataset["EDesigNum"]
del dataset["APPDate"]
del dataset["FIRM07_FLA"]
del dataset["PFIRM15_FL"]
del dataset["Borough"]
del dataset["Version"]
del dataset["BBL"]
del dataset["APPBBL"]

*Заменяю промпущенные значение номинативных признаков на моду числа:*

In [7]:
dataset.neighborhood.fillna(dataset.neighborhood.mode()[0],inplace=True)

In [8]:
dataset.building_class_category.fillna(dataset.building_class_category.mode()[0],inplace=True)

In [9]:
dataset.tax_class.fillna(dataset.tax_class.mode()[0],inplace=True)

In [10]:
dataset.building_class.fillna(dataset.building_class.mode()[0],inplace=True)

In [11]:
dataset.address.fillna(dataset.address.mode()[0],inplace=True)

In [12]:
dataset.building_class_at_sale.fillna(dataset.building_class_at_sale.mode()[0],inplace=True)

In [13]:
dataset.sale_date.fillna(dataset.sale_date.mode()[0],inplace=True)

In [14]:
dataset.FireComp.fillna(dataset.FireComp.mode()[0],inplace=True)

In [15]:
dataset.SanitSub.fillna(dataset.SanitSub.mode()[0],inplace=True)

In [16]:
dataset.Address.fillna(dataset.Address.mode()[0],inplace=True)

In [17]:
dataset.ZoneDist1.fillna(dataset.ZoneDist1.mode()[0],inplace=True)

In [18]:
dataset.SplitZone.fillna(dataset.SplitZone.mode()[0],inplace=True)

In [19]:
dataset.BldgClass.fillna(dataset.BldgClass.mode()[0],inplace=True)

In [20]:
dataset.OwnerName.fillna(dataset.OwnerName.mode()[0],inplace=True)

In [21]:
dataset.IrrLotCode.fillna(dataset.IrrLotCode.mode()[0],inplace=True)

In [22]:
dataset.ZoneMap.fillna(dataset.ZoneMap.mode()[0],inplace=True)

In [23]:
dataset.TaxMap.fillna(dataset.TaxMap.mode()[0],inplace=True)

*Пропущенные значения в количественных признаков заменяю на медианное значение:*

In [24]:
dataset.fillna(dataset.median(),inplace=True)

*Перевожу все значения номинативных признаков в числа с помощью библиотеrb $LabelEncoder$ и словаря*

In [25]:
le=LabelEncoder()
dataset["ZoneMap"]= le.fit_transform(dataset["ZoneMap"].astype(str))
dataset["building_class_category"]= le.fit_transform(dataset["building_class_category"].astype(str))
dataset["neighborhood"]= le.fit_transform(dataset["neighborhood"].astype(str))
dataset["year_of_sale"]= le.fit_transform(dataset["year_of_sale"].astype(str))
dataset["ZoneDist1"]= le.fit_transform(dataset["ZoneDist1"].astype(str))
dataset["BldgClass"]= le.fit_transform(dataset["BldgClass"].astype(str))
dataset["SanitSub"]= le.fit_transform(dataset["SanitSub"].astype(str))
dataset["building_class"]= le.fit_transform(dataset["building_class"].astype(str))
dataset["FireComp"]= le.fit_transform(dataset["FireComp"].astype(str))
dataset["building_class_at_sale"]= le.fit_transform(dataset["building_class_at_sale"].astype(str))
dataset["tax_class_at_sale"]= le.fit_transform(dataset["tax_class_at_sale"].astype(str))
dataset["tax_class"]= le.fit_transform(dataset["tax_class"].astype(str))
dataset["address"]= le.fit_transform(dataset["address"].astype(str))
dataset["Address"]= le.fit_transform(dataset["Address"].astype(str))
dataset["OwnerName"]= le.fit_transform(dataset["OwnerName"].astype(str))
dataset["sale_date"]= le.fit_transform(dataset["sale_date"].astype(str))
dataset["Sanborn"]= le.fit_transform(dataset["Sanborn"].astype(str))

In [26]:
d = {"N": 0, "Y":1}
q = {"False": 0, "True":1}
dataset["IrrLotCode"]= dataset["IrrLotCode"].map(d)
dataset["SplitZone"]= dataset["SplitZone"].map(d)
dataset=(dataset.astype('int'))

*Нормализую признаки, что бы они были в одном диапазоне*

In [28]:
num_rows = len(dataset.index)
low_information_cols = [] 

for col in dataset.columns:
    cnts = dataset[col].value_counts(dropna=True)
    top_pct = (cnts/num_rows).iloc[0]
    
    if top_pct > 0.96:
        del dataset[col]
        low_information_cols.append(col)
        print('{0}: {1:.5f}%'.format(col, top_pct*100))
        print(cnts)
        print()
        

SanitBoro: 99.97314%
3    390778
4       101
2         2
1         1
5         1
Name: SanitBoro, dtype: int64

SplitZone: 96.12442%
0    375734
1     15149
Name: SplitZone, dtype: int64

Easements: 99.46966%
0    388810
1      2005
2        68
Name: Easements, dtype: int64

OfficeArea: 97.93442%
0        382809
336         338
2000        296
1000        236
1500        181
          ...  
54932         1
1934          1
144           1
2193          1
1805          1
Name: OfficeArea, Length: 1649, dtype: int64

GarageArea: 98.99433%
0         386952
157343       267
34600        237
2000         155
11452        133
           ...  
8300           1
5550           1
1195           1
2409           1
48782          1
Name: GarageArea, Length: 792, dtype: int64

StrgeArea: 99.44868%
0       388728
2500        70
5000        67
4000        55
2000        49
         ...  
1440         1
1184         1
845          1
1950         1
2938         1
Name: StrgeArea, Length: 713, dtype: int

### ОТБОР ПРИЗНАКОВ

In [29]:
dataset.corr()

,neighborhood,building_class_category,tax_class,block,lot,building_class,address,zip_code,residential_units,commercial_units,...,CommFAR,FacilFAR,Tract2010,XCoord,YCoord,ZoneMap,Sanborn,TaxMap,SHAPE_Leng,SHAPE_Area
neighborhood,1.000000,0.094541,0.094772,-0.045158,0.100379,0.080687,-0.031200,0.002962,-0.003098,0.005127,...,-0.001187,-0.083331,0.012833,-0.012150,-0.066500,0.023763,0.128776,-0.028196,0.023715,0.007377
building_class_category,0.094541,1.000000,0.826846,-0.179083,0.410429,0.553936,0.040186,-0.031814,-0.004845,0.122148,...,0.157981,0.021446,-0.080265,-0.058268,0.035015,-0.260789,0.234223,-0.301922,0.108269,0.076911
tax_class,0.094772,0.826846,1.000000,-0.219682,0.361756,0.526411,0.041381,-0.006405,0.007825,0.142674,...,0.176511,0.104783,-0.082261,-0.057078,0.067394,-0.280291,0.134448,-0.312353,0.168103,0.118596
block,-0.045158,-0.179083,-0.219682,1.000000,-0.096718,-0.176236,-0.124946,0.011148,-0.008497,-0.020956,...,-0.123140,-0.249097,0.137176,-0.000427,-0.500335,0.739987,0.453328,0.810059,0.071903,0.052157
lot,0.100379,0.410429,0.361756,-0.096718,1.000000,0.669580,-0.026449,-0.017627,-0.015153,-0.022427,...,-0.080647,-0.222720,-0.078160,0.014586,-0.019208,-0.228810,0.587408,-0.349588,-0.110177,-0.084003
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZoneMap,0.023763,-0.260789,-0.280291,0.739987,-0.228810,-0.286774,-0.076971,0.013071,-0.004677,-0.031289,...,-0.116110,-0.230682,0.130785,-0.019249,-0.634244,1.000000,0.361842,0.821054,0.114634,0.088115
Sanborn,0.128776,0.234223,0.134448,0.453328,0.587408,0.393692,-0.067122,-0.010961,-0.012487,-0.038017,...,-0.166630,-0.360441,0.018851,0.040129,-0.403432,0.361842,1.000000,0.270888,-0.035703,-0.027941
TaxMap,-0.028196,-0.301922,-0.312353,0.810059,-0.349588,-0.364301,-0.094928,0.013241,-0.001833,-0.020612,...,-0.103208,-0.174149,0.173489,0.013933,-0.485625,0.821054,0.270888,1.000000,0.101493,0.074572
SHAPE_Leng,0.023715,0.108269,0.168103,0.071903,-0.110177,-0.069427,-0.011798,-0.002449,0.014395,0.018144,...,0.032392,0.162623,0.044689,-0.010027,-0.071470,0.114634,-0.035703,0.101493,1.000000,0.887185


*Корреляция признака $building class category$ хорошая*

*Из графика можно сделать вывод, что все признаки подходят для обучение модели по классификации.*

### РАЗБИЕНИЕ ОБРАБОТАННЫХ ДАННЫХ НА ОБУЧАЮЩУЮ И ТЕСТИРУЮЩУЮ ВЫБОРКИ

*Нормализую признаки, что бы они были в одном диапазоне*

In [30]:
scaler=MinMaxScaler()

In [31]:
df_scaler=scaler.fit_transform(dataset.drop("building_class_category",axis=1))

In [32]:
pd.DataFrame(df_scaler,columns=dataset.drop("building_class_category",axis=1).columns)

,neighborhood,tax_class,block,lot,building_class,address,zip_code,residential_units,commercial_units,total_units,...,CommFAR,FacilFAR,Tract2010,XCoord,YCoord,ZoneMap,Sanborn,TaxMap,SHAPE_Leng,SHAPE_Area
0,0.396825,1.0,0.015524,0.109505,0.703297,0.578817,0.331361,0.000000,0.002817,0.000048,...,0.000000,0.166667,0.003722,0.974022,0.844054,0.290323,1.000000,0.938807,0.012438,0.000324
1,0.365079,1.0,0.005919,0.000000,0.285714,0.922132,0.331361,0.000000,0.000000,0.000000,...,0.166667,0.500000,0.000178,0.964857,0.935726,0.032258,0.047341,0.923195,0.078403,0.019601
2,0.158730,1.0,0.022671,0.000000,0.571429,0.527334,0.331361,0.000000,0.000000,0.000000,...,0.166667,0.000000,0.000008,0.962654,0.935707,0.032258,0.039559,0.923348,0.044715,0.004847
3,0.698413,1.0,0.945834,0.005913,0.445055,0.726883,0.332337,0.000000,0.346479,0.005910,...,0.166667,0.000000,0.005905,0.982811,0.774565,0.645161,0.984436,0.996841,0.187673,0.111556
4,0.158730,1.0,0.025575,0.000000,0.340659,0.375322,0.331361,0.000000,0.002817,0.000048,...,0.000000,0.333333,0.002546,0.962332,0.929498,0.032258,0.046044,0.923348,0.031119,0.002988
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
390878,1.000000,0.0,0.372571,0.001971,0.071429,0.611027,0.332426,0.000144,0.000000,0.000144,...,0.000000,0.333333,0.003714,0.984636,0.932233,0.096774,0.420233,0.954021,0.012186,0.000350
390879,1.000000,0.0,0.372571,0.001971,0.071429,0.611027,0.332426,0.000144,0.000000,0.000144,...,0.000000,0.333333,0.003714,0.984636,0.932233,0.096774,0.420233,0.954021,0.012186,0.000350
390880,1.000000,1.0,0.366205,0.005366,1.000000,0.626036,0.332426,0.000000,0.000000,0.000000,...,0.000000,0.333333,0.003748,0.983372,0.935515,0.096774,0.418936,0.953960,0.011531,0.000391
390881,0.000000,0.0,0.768930,0.121113,0.071429,0.155802,0.331746,0.000000,0.000000,0.000000,...,0.000000,0.166667,0.003722,0.974022,0.844054,0.290323,1.000000,0.938807,0.012438,0.000324


In [33]:
x=df_scaler.copy()
y=dataset["building_class_category"]

In [34]:
x_train, x_test, y_train, y_test=train_test_split(x,y,test_size=0.4, random_state=20)

### КЛАССИФИФКАЦИЯ ОБЪЕКТОВ

In [35]:
log=LogisticRegression(random_state=10, C=1.0, n_jobs= -1)

In [36]:
log.fit(x_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=-1, penalty='l2', random_state=10,
                   solver='lbfgs', tol=0.0001, verbose=0, warm_start=False)

In [37]:
y_pred=log.predict(x_test)

In [38]:
log.score(x_test, y_test)

0.7768077567571025

In [39]:
print(classification_report(y_pred, y_test))

              precision    recall  f1-score   support

           0       0.40      0.67      0.50     13038
           1       0.90      0.74      0.81     51144
           2       0.87      0.87      0.87     15464
           3       0.97      0.91      0.94      3786
           4       0.00      0.03      0.01        29
           5       0.90      0.86      0.88      3034
           6       0.03      0.07      0.04        42
           7       0.01      0.02      0.01       110
           8       0.00      0.00      0.00         0
           9       0.90      0.85      0.88     12191
          10       0.29      0.68      0.40       241
          11       0.07      0.46      0.12       123
          12       0.00      0.00      0.00         0
          13       0.53      0.74      0.62      2330
          14       0.02      0.19      0.04        27
          15       0.97      0.88      0.92     13908
          16       0.71      0.56      0.63      1041
          17       0.00    

In [40]:
sgd=SGDClassifier(random_state=1,epsilon=0.01)

In [41]:
sgd.fit(x_train, y_train)

SGDClassifier(alpha=0.0001, average=False, class_weight=None,
              early_stopping=False, epsilon=0.01, eta0=0.0, fit_intercept=True,
              l1_ratio=0.15, learning_rate='optimal', loss='hinge',
              max_iter=1000, n_iter_no_change=5, n_jobs=None, penalty='l2',
              power_t=0.5, random_state=1, shuffle=True, tol=0.001,
              validation_fraction=0.1, verbose=0, warm_start=False)

In [42]:
y_pred=sgd.predict(x_test)

In [43]:
sgd.score(x_test, y_test)

0.7463768115942029

In [44]:
print(classification_report(y_pred, y_test))

              precision    recall  f1-score   support

           0       0.17      0.81      0.29      4659
           1       0.95      0.69      0.80     58173
           2       0.94      0.89      0.92     16273
           3       0.82      0.85      0.83      3404
           4       0.00      0.00      0.00         0
           5       0.89      0.82      0.85      3115
           6       0.00      0.00      0.00         0
           7       0.00      0.00      0.00         0
           8       0.00      0.00      0.00         1
           9       0.85      0.84      0.84     11597
          10       0.01      0.38      0.01         8
          11       0.24      0.13      0.17      1553
          12       0.00      0.00      0.00         0
          13       0.54      0.68      0.61      2562
          14       0.03      0.60      0.05        10
          15       0.99      0.83      0.90     15106
          16       0.01      0.11      0.02        62
          17       0.00    

In [45]:
tree=DecisionTreeClassifier(random_state=1)

In [46]:
tree.fit(x_train, y_train)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=1, splitter='best')

In [47]:
y_pred=tree.predict(x_test)

In [46]:
tree.score(x_test,y_test)

0.9995906724484184

In [47]:
print(classification_report(y_pred, y_test))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00     21678
           1       1.00      1.00      1.00     42498
           2       1.00      1.00      1.00     15393
           3       1.00      1.00      1.00      3541
           4       1.00      1.00      1.00       226
           5       1.00      1.00      1.00      2878
           6       1.00      1.00      1.00       104
           7       0.99      1.00      1.00       352
           8       0.93      1.00      0.96        13
           9       1.00      1.00      1.00     11481
          10       1.00      1.00      1.00       571
          11       1.00      1.00      1.00       858
          12       0.93      0.93      0.93        30
          13       1.00      1.00      1.00      3225
          14       1.00      1.00      1.00       223
          15       1.00      1.00      1.00     12644
          16       1.00      1.00      1.00       823
          17       0.00    

*Наиболее точное предсказание при обучении у классификатора $Decision Tree Regressor$*

### ОТБОР ПРИЗНАКОВ

In [48]:
dataset.corr()

,neighborhood,building_class_category,tax_class,block,lot,building_class,address,zip_code,residential_units,commercial_units,...,Tract2010,XCoord,YCoord,ZoneMap,Sanborn,TaxMap,PLUTOMapID,MAPPLUTO_F,SHAPE_Leng,SHAPE_Area
neighborhood,1.000000,0.094541,0.094772,-0.045158,0.100379,0.080687,-0.031200,0.002962,-0.003098,0.005127,...,0.012833,-0.012150,-0.066500,0.023763,0.128776,-0.028196,-0.004760,-0.000140,0.023715,0.007377
building_class_category,0.094541,1.000000,0.826846,-0.179083,0.410429,0.553936,0.040186,-0.031814,-0.004845,0.122148,...,-0.080265,-0.058268,0.035015,-0.260789,0.234223,-0.301922,0.001701,0.000853,0.108269,0.076911
tax_class,0.094772,0.826846,1.000000,-0.219682,0.361756,0.526411,0.041381,-0.006405,0.007825,0.142674,...,-0.082261,-0.057078,0.067394,-0.280291,0.134448,-0.312353,0.000982,0.002316,0.168103,0.118596
block,-0.045158,-0.179083,-0.219682,1.000000,-0.096718,-0.176236,-0.124946,0.011148,-0.008497,-0.020956,...,0.137176,-0.000427,-0.500335,0.739987,0.453328,0.810059,0.006643,-0.000751,0.071903,0.052157
lot,0.100379,0.410429,0.361756,-0.096718,1.000000,0.669580,-0.026449,-0.017627,-0.015153,-0.022427,...,-0.078160,0.014586,-0.019208,-0.228810,0.587408,-0.349588,0.001894,0.038355,-0.110177,-0.084003
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TaxMap,-0.028196,-0.301922,-0.312353,0.810059,-0.349588,-0.364301,-0.094928,0.013241,-0.001833,-0.020612,...,0.173489,0.013933,-0.485625,0.821054,0.270888,1.000000,-0.184695,-0.015931,0.101493,0.074572
PLUTOMapID,-0.004760,0.001701,0.000982,0.006643,0.001894,0.004590,0.005200,0.000346,-0.000288,-0.000790,...,-0.001813,-0.114304,-0.081119,0.007475,0.001380,-0.184695,1.000000,0.315847,0.000301,-0.000387
MAPPLUTO_F,-0.000140,0.000853,0.002316,-0.000751,0.038355,0.006850,0.001428,0.000194,-0.000239,-0.000616,...,-0.001286,-0.050479,-0.035997,0.000454,-0.003469,-0.015931,0.315847,1.000000,0.016103,0.015674
SHAPE_Leng,0.023715,0.108269,0.168103,0.071903,-0.110177,-0.069427,-0.011798,-0.002449,0.014395,0.018144,...,0.044689,-0.010027,-0.071470,0.114634,-0.035703,0.101493,0.000301,0.016103,1.000000,0.887185


*Корреляция признака $sale price$ хорошая*

*все признаки подходят для прогнозирования*

### РАЗБИЕНИЕ ДАННЫХ НА ОБУЧАЮЩУЮ И ТЕСТИРУЮЩУЮ ВЫБОРКИ

*Нормализую признаки, что бы они были в одном диапазоне*

In [49]:
scaler=MinMaxScaler()

In [50]:
df_scaler=scaler.fit_transform(dataset.drop("sale_price",axis=1))

In [51]:
pd.DataFrame(df_scaler,columns=dataset.drop("sale_price",axis=1).columns)

,neighborhood,building_class_category,tax_class,block,lot,building_class,address,zip_code,residential_units,commercial_units,...,Tract2010,XCoord,YCoord,ZoneMap,Sanborn,TaxMap,PLUTOMapID,MAPPLUTO_F,SHAPE_Leng,SHAPE_Area
0,0.396825,0.517241,1.0,0.015524,0.109505,0.703297,0.578817,0.331361,0.000000,0.002817,...,0.003722,0.974022,0.844054,0.290323,1.000000,0.938807,0.0,0.0,0.012438,0.000324
1,0.365079,0.540230,1.0,0.005919,0.000000,0.285714,0.922132,0.331361,0.000000,0.000000,...,0.000178,0.964857,0.935726,0.032258,0.047341,0.923195,0.0,0.0,0.078403,0.019601
2,0.158730,0.390805,1.0,0.022671,0.000000,0.571429,0.527334,0.331361,0.000000,0.000000,...,0.000008,0.962654,0.935707,0.032258,0.039559,0.923348,0.0,0.0,0.044715,0.004847
3,0.698413,0.413793,1.0,0.945834,0.005913,0.445055,0.726883,0.332337,0.000000,0.346479,...,0.005905,0.982811,0.774565,0.645161,0.984436,0.996841,0.0,0.0,0.187673,0.111556
4,0.158730,0.482759,1.0,0.025575,0.000000,0.340659,0.375322,0.331361,0.000000,0.002817,...,0.002546,0.962332,0.929498,0.032258,0.046044,0.923348,0.0,0.0,0.031119,0.002988
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
390878,1.000000,0.586207,0.0,0.372571,0.001971,0.071429,0.611027,0.332426,0.000144,0.000000,...,0.003714,0.984636,0.932233,0.096774,0.420233,0.954021,0.0,0.0,0.012186,0.000350
390879,1.000000,0.586207,0.0,0.372571,0.001971,0.071429,0.611027,0.332426,0.000144,0.000000,...,0.003714,0.984636,0.932233,0.096774,0.420233,0.954021,0.0,0.0,0.012186,0.000350
390880,1.000000,0.804598,1.0,0.366205,0.005366,1.000000,0.626036,0.332426,0.000000,0.000000,...,0.003748,0.983372,0.935515,0.096774,0.418936,0.953960,0.0,0.0,0.011531,0.000391
390881,0.000000,0.034483,0.0,0.768930,0.121113,0.071429,0.155802,0.331746,0.000000,0.000000,...,0.003722,0.974022,0.844054,0.290323,1.000000,0.938807,0.0,0.0,0.012438,0.000324


In [52]:
x=df_scaler.copy()
y=dataset["sale_price"]

In [53]:
x_train, x_test, y_train, y_test=train_test_split(x,y,test_size=0.4, random_state=20)

### ПРОГНОЗИРОВАНИЕ

In [ ]:
sgd=SGDClassifier(random_state=1,epsilon=0.01)

In [ ]:
sgd.fit(x_train, y_train)

In [ ]:
y_pred = sgd.predict(x_test)

In [ ]:
sgd.score(x_test, y_test)

In [ ]:
accuracy_score(y_test, y_pred)

In [ ]:
print(classification_report(y_test,y_pred))

### РАЗРАБОТКА ЧАТ-БОТА

### ПРЕДОБРАБОТКА ДАННЫХ

In [ ]:
a_dict = {2003:1, 2004:2, 2005:3, 2006:4, 2007:5, 2008:6, 2009:7, 2010:8, 2011:9,
          2012:10, 2013:11, 2014:12, 2015:13, 2016:14, 2017:15}
 
for key in a_dict:
    print(key,"y=",round(dataset[(dataset["year_of_sale"] ==key)]["sale_price"].mean(),0),"$")